In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import time
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from pylab import rcParams
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import NearMiss
from collections import Counter
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from imblearn.under_sampling import NearMiss
import sklearn.linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
df = pd.read_csv("heart3.csv")
df

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,40,4.0,0,1.0,0.0,0,0,0,196.0,107.0,71.0,27.97,81.0,78.0,0
1,0,47,2.0,0,1.0,0.0,0,0,0,251.0,122.0,82.0,29.73,96.0,77.0,0
2,1,49,1.0,1,21.0,0.0,0,0,0,246.0,128.5,81.0,26.34,76.0,71.0,0
3,0,62,3.0,1,31.0,0.0,0,1,0,226.0,151.0,96.0,29.58,66.0,79.0,1
4,0,47,3.0,1,24.0,0.0,0,0,0,286.0,131.0,85.0,24.10,86.0,86.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7183,1,62,1.0,1,21.0,0.0,0,1,0,221.0,216.0,130.0,25.69,93.0,79.0,1
7184,1,52,1.0,1,8.0,0.0,0,1,0,134.0,139.0,79.0,17.98,81.0,66.0,1
7185,0,56,4.0,0,1.0,0.0,0,1,0,261.0,173.5,101.5,33.27,59.0,73.0,1
7186,0,52,2.0,0,1.0,0.0,0,0,0,285.0,133.0,79.0,22.94,69.0,95.0,1


# Feature selection

In [ ]:
df3 = df.drop(['education','currentSmoker'],axis=1)
df3

,male,age,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,40,1.0,0.0,0,0,0,196.0,107.0,71.0,27.97,81.0,78.0,0
1,0,47,1.0,0.0,0,0,0,251.0,122.0,82.0,29.73,96.0,77.0,0
2,1,49,21.0,0.0,0,0,0,246.0,128.5,81.0,26.34,76.0,71.0,0
3,0,62,31.0,0.0,0,1,0,226.0,151.0,96.0,29.58,66.0,79.0,1
4,0,47,24.0,0.0,0,0,0,286.0,131.0,85.0,24.10,86.0,86.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7183,1,62,21.0,0.0,0,1,0,221.0,216.0,130.0,25.69,93.0,79.0,1
7184,1,52,8.0,0.0,0,1,0,134.0,139.0,79.0,17.98,81.0,66.0,1
7185,0,56,1.0,0.0,0,1,0,261.0,173.5,101.5,33.27,59.0,73.0,1
7186,0,52,1.0,0.0,0,0,0,285.0,133.0,79.0,22.94,69.0,95.0,1


In [ ]:
#df3.to_csv("heart4.csv", index=False)